In [1]:
import pandas as pd
import numpy as np

In [2]:
wa_train = pd.read_csv('../../trainData/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})
wa_test = pd.read_csv('../../testData_B/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [3]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})

In [5]:
uid_train = pd.read_csv('../../trainData/uid_train.txt',sep='\t',header=None,names=('uid','label'))

In [6]:
wa = pd.concat([wa_train,wa_test],axis=0)
uid = pd.concat([uid_train, uid_test], axis=0)

In [9]:
# calculate up_flow_per_visit, down_flow_per_visit, dura_per_visit
wa['up_flow_per_visit'] = wa['up_flow'] / wa['visit_cnt']
wa['down_flow_per_visit'] = wa['down_flow'] / wa['visit_cnt']
wa['dura_per_visit'] = wa['visit_dura'] / wa['visit_cnt']

In [31]:
wa_up_flow_per_visit = wa.groupby(['uid'])['up_flow_per_visit'].agg(['std','max','min','median','mean']).add_prefix('wa_up_flow_per_visit_').reset_index()
wa_down_flow_per_visit = wa.groupby(['uid'])['down_flow_per_visit'].agg(['std','max','min','median','mean']).add_prefix('wa_down_flow_per_visit_').reset_index()
wa_dura_per_visit = wa.groupby(['uid'])['dura_per_visit'].agg(['std','max','min','median','mean']).add_prefix('wa_dura_per_visit_').reset_index()


In [34]:
pd.isnull(wa_up_flow_per_visit).any()

uid                            False
wa_up_flow_per_visit_std        True
wa_up_flow_per_visit_max        True
wa_up_flow_per_visit_min        True
wa_up_flow_per_visit_median     True
wa_up_flow_per_visit_mean       True
dtype: bool

In [35]:
feat_wa_per_visit = uid

features = [wa_up_flow_per_visit, wa_down_flow_per_visit, wa_dura_per_visit]

for feat in features:
    feat_wa_per_visit = pd.merge(feat_wa_per_visit, feat, how='left', on='uid')

feat_wa_per_visit.to_csv('../myFeatures/feat_wa_per_visit.csv')

In [43]:
feat_wa_per_visit.columns

Index(['label', 'uid', 'wa_up_flow_per_visit_std', 'wa_up_flow_per_visit_max',
       'wa_up_flow_per_visit_min', 'wa_up_flow_per_visit_median',
       'wa_up_flow_per_visit_mean', 'wa_down_flow_per_visit_std',
       'wa_down_flow_per_visit_max', 'wa_down_flow_per_visit_min',
       'wa_down_flow_per_visit_median', 'wa_down_flow_per_visit_mean',
       'wa_dura_per_visit_std', 'wa_dura_per_visit_max',
       'wa_dura_per_visit_min', 'wa_dura_per_visit_median',
       'wa_dura_per_visit_mean'],
      dtype='object')

# Feat 2

In [24]:
def formDiffDataFrame(df, prefix):
    df_result = pd.DataFrame()
    cols = ['std', 'max', 'min', 'median', 'mean', 'sum']
    
    first_time = True
    for col in cols:
        t = df.unstack()[:][col].add_prefix(prefix + col + '_in_wa_type_').reset_index()
        if first_time:
            df_result = t
            first_time = False
        else:
            df_result = pd.merge(df_result, t, how='left', on='uid')
    
    return df_result

In [28]:
# 计算每种wa_type该用户的visit_cnt, visit_dura, up_flow, down_flow
wa_type_info_visit_cnt = wa.groupby(['uid', 'wa_type'])['visit_cnt'].agg(['std','max','min','median','mean','sum'])
wa_type_info_visit_dura = wa.groupby(['uid', 'wa_type'])['visit_dura'].agg(['std','max','min','median','mean','sum'])
wa_type_info_up_flow = wa.groupby(['uid', 'wa_type'])['up_flow'].agg(['std','max','min','median','mean','sum'])
wa_type_info_down_flow = wa.groupby(['uid', 'wa_type'])['down_flow'].agg(['std','max','min','median','mean','sum'])

In [29]:
wa_type_info_visit_cnt = formDiffDataFrame(wa_type_info_visit_cnt, 'visit_cnt_')
wa_type_info_visit_dura = formDiffDataFrame(wa_type_info_visit_dura, 'visit_dura_')
wa_type_info_up_flow = formDiffDataFrame(wa_type_info_up_flow, 'up_flow_')
wa_type_info_down_flow = formDiffDataFrame(wa_type_info_down_flow, 'down_flow_')

In [36]:
feat_wa_type_info = uid

features = [wa_type_info_visit_cnt, wa_type_info_visit_dura, wa_type_info_up_flow, wa_type_info_down_flow]

for feat in features:
    feat_wa_type_info = pd.merge(feat_wa_type_info, feat, how='left', on='uid')

feat_wa_type_info.to_csv('../myFeatures/feat_wa_type_info.csv')

# 拼接

In [45]:
feat_all = feat_wa_per_visit[['label', 'uid', 'wa_up_flow_per_visit_median', 'wa_down_flow_per_visit_median', 'wa_dura_per_visit_median']]
# feat_all = pd.merge(feat_all, feat_wa_type_info.drop(['label'], axis=1), how='left', on='uid')


In [46]:
train_features = feat_all[feat_all['uid'].isin(uid_train['uid'])]
train_features.shape

(4999, 5)

In [47]:
test_features = feat_all[feat_all['uid'].isin(uid_test['uid'])]
test_features.shape

(3000, 5)

In [48]:
train_features.to_csv('../myFeatures/train_features3.csv',index=None)
test_features.to_csv('../myFeatures/test_features3.csv',index=None)